# Amazon Sales EDA
Dataset: https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset

In [2]:
import pandas as pd
import re

In [3]:
sales = pd.read_csv("amazon.csv")
sales

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,₹379,₹919,59%,4,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,"₹2,280","₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the me

In [4]:
len(sales['product_name'].unique())

1337

todo some EDA

## Create MySQL DDL

```sql
CREATE TABLE product
(
    id             VARCHAR(20)  NOT NULL,
    sku            VARCHAR(255) NULL,
    name           VARCHAR(500) NULL,
    `description`  TEXT         NULL,
    price          DECIMAL      NULL,
    `image_url`    VARCHAR(255) NULL,
    stock_quantity INT          NULL,
    created_at     datetime     NULL,
    updated_at     datetime     NULL,
    CONSTRAINT pk_product PRIMARY KEY (id)
);

CREATE TABLE orders
(
    id           BIGINT       NOT NULL AUTO_INCREMENT,
    customer_id  VARCHAR(255) NULL,
    total_amount DECIMAL      NULL,
    status       SMALLINT     NULL,
    created_at   datetime     NULL,
    CONSTRAINT pk_order PRIMARY KEY (id)
);

CREATE TABLE order_item
(
    id         BIGINT  NOT NULL AUTO_INCREMENT,
    product_id VARCHAR(20)  NULL,
    order_id   BIGINT  NULL,
    quantity   DECIMAL NULL,
    unit_price DECIMAL NULL,
    CONSTRAINT pk_order_item PRIMARY KEY (id)
);

ALTER TABLE order_item
ADD CONSTRAINT fk_order_item_on_order
FOREIGN KEY (order_id) REFERENCES orders (id);


ALTER TABLE order_item
ADD CONSTRAINT fk_order_item_on_product 
FOREIGN KEY (product_id) REFERENCES product (id);
```

## Enrich Inventory with Amazon Products

**Load Custom MySQL Connector**

In [5]:
from scripts.connector import get_connector

Show tables in the database:

In [6]:
cnx = get_connector()
cursor = cnx.cursor()

cursor.execute("SHOW TABLES;")

tables = list()

for table in cursor.fetchall():
    tables.append(table[0])

cursor.close()
cnx.close()
tables

['order', 'order_item', 'product']

### Create Products Dataset

In [7]:
products = sales[['product_id', 'product_name', 'about_product', 'actual_price', 'img_link']]
products.columns

Index(['product_id', 'product_name', 'about_product', 'actual_price',
       'img_link'],
      dtype='object')

Create missing columns for `product` table and set stock quantity as $50000$ items per each.

In [8]:
products = products.drop_duplicates(subset=["product_id"], ignore_index=True)
products = products.rename(columns={"product_id": "id", "product_name": "name", "about_product": "description", "actual_price": "price", "img_link": "image_url"})
products["sku"] = "SKU-" + products["id"]
products.price = (
    products.price
    .astype(str)
    .apply(lambda x: int("".join(re.findall(r"\d+", x))) if x != "nan" else None)
)
# set some default for each product
products["stock_quantity"] = 50000
now = pd.Timestamp.now()

products["created_at"] = now
products["updated_at"] = now
products

,id,name,description,price,image_url,sku,stock_quantity,created_at,updated_at
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,High Compatibility : Compatible With iPhone 12...,1099,https://m.media-amazon.com/images/W/WEBP_40237...,SKU-B07JW9H4J1,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,"Compatible with all Type C enabled devices, be...",349,https://m.media-amazon.com/images/W/WEBP_40237...,SKU-B098NS6PVG,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,【 Fast Charger& Data Sync】-With built-in safet...,1899,https://m.media-amazon.com/images/W/WEBP_40237...,SKU-B096MSW6CT,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,The boAt Deuce USB 300 2 in 1 cable is compati...,699,https://m.media-amazon.com/images/I/41V5FtEWPk...,SKU-B08HDJ86NZ,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,399,https://m.media-amazon.com/images/W/WEBP_40237...,SKU-B08CF3B7N1,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438
...,...,...,...,...,...,...,...,...,...
1346,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,919,https://m.media-amazon.com/images/I/41fDdRtjfx...,SKU-B08L7J3T31,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438
1347,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,"230 Volts, 400 watts, 1 Year",3045,https://m.media-amazon.com/images/I/41gzDxk4+k...,SKU-B01M6453MB,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438
1348,B009P2LIL4,Bajaj Majesty RX10 2000 Watts Heat Convector R...,International design and styling|Two heat sett...,3080,https://m.media-amazon.com/images/W/WEBP_40237...,SKU-B009P2LIL4,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438
1349,B00J5DYCCA,Havells Ventil Air DSP 230mm Exhaust Fan (Pist...,Fan sweep area: 230 MM ; Noise level: (40 - 45...,1890,https://m.media-amazon.com/images/W/WEBP_40237...,SKU-B00J5DYCCA,50000,2026-01-05 18:26:48.189438,2026-01-05 18:26:48.189438


### Write Products into Database

In [9]:
cnx = get_connector()
cursor = cnx.cursor()

table = "product"

columns = [
    "id",
    "name",
    "description",
    "price",
    "image_url",
    "sku",
    "stock_quantity",
    "created_at",
    "updated_at"
]

cols_sql = ", ".join(f"`{c}`" for c in columns)
placeholders = ", ".join(["%s"] * len(columns))

sql = f"""
INSERT INTO `{table}` ({cols_sql})
VALUES ({placeholders})
"""

data = list(products[columns].itertuples(index=False, name=None))

cursor.executemany(sql, data)
cnx.commit()

cursor.close()
cnx.close()